# Exercise 22 Solution - Singular Value Decomposition

### Task
Implement the singular value decomposition as function and apply it to three examples

### Learning goals
- Understand and be able to compute a singular value decomposition
- Familiarize yourself with its effect on linear dependent matrices

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Singular value decomposition implementation

decompose as
$$\boldsymbol{A}=\boldsymbol{U}\boldsymbol{\Sigma}\boldsymbol{V}^\intercal$$
such that 
$$\boldsymbol{U}^\intercal \boldsymbol{U}=\boldsymbol{I}$$
$$\boldsymbol{V}^\intercal \boldsymbol{V}=\boldsymbol{I}$$
and
$$\boldsymbol{\Sigma}=\boldsymbol{U}^\intercal \boldsymbol{A} \boldsymbol{V}$$

hint: use `np.linalg.eig`

In [ ]:
def SVD(A):
    lu, U = np.linalg.eig(np.dot(A, np.transpose(A)))
    lu[np.abs(lu) < 1e-10] = 0.
    index = np.argsort(lu)[::-1]
    lu = lu[index]
    U = U[:, index]

    lv, V = np.linalg.eig(np.dot(np.transpose(A), A))
    lv[np.abs(lv) < 1e-10] = 0.
    index = np.argsort(lv)[::-1]
    lv = lv[index]
    V = V[:, index]

    S = np.zeros((len(U), len(V)))
    for i in range(min(len(U), len(V))):
        S[i, i] = np.sqrt(lu[i])

    # correction to make U and V consistent
    S_correct = np.dot(np.dot(np.transpose(U), A), (V))
    for i in range(min(len(U), len(V))):
        V[:, i] = V[:, i] * np.sign(S_correct[i, i])
    return U, S, V

## Singular value decomposition

### example 1 (3x3) matrix

**decomposition**

In [ ]:
A = np.array([[3, 2, 2], [2, 3, -2]])
U, S, V = SVD(A)

**reconstruction**

In [ ]:
AReconstruct = np.dot(np.dot(U, S), np.transpose(V))
print(AReconstruct)

### example 2 (5x5) matrix (with linear dependencies)

**decomposition**

In [ ]:
A = np.array([[1, 0, 0, 2, 0],
              [0, 0, 0, 0, 1],
              [3, 4, 1, 1, 2],
              [2, 0, 0, 4, 0],
              [0, 0, 0, 0, 2]])
U, S, V = SVD(A)

In [ ]:
print(S)

### example 3 image as matrix

**load image**

In [ ]:
img = Image.open('GuentherSmall.jpg')
data = np.asarray(img)

**convert image to grayscale image**

In [ ]:
rgb_weights = [0.2989, 0.5870, 0.1140]
data = np.dot(data[..., :3], rgb_weights)

**singular value decomposition**

In [ ]:
U, S, V = np.linalg.svd(data)
# U is orthogonal and an orthonormal basis for column space of data
# S is diagonal: singular values in descending order
# V is orthogonal: orthonormal basis for row space of data

**truncation**

In [ ]:
r = 20  # truncation level
dataReconstruct = U[:, :r] @ np.diag(S[:r]) @ V[:r, :]

rows, cols = data.shape

print(f"Original image size: {rows} x {cols} = {rows*cols} pixels")
print(f"Reduced image size: {rows} x {r} + {r} x {r} + {cols} x {r} = {rows*r + r*r + cols*r} pixels")
print(f"Compressed size: {(100*(rows*r + r*r + cols*r) / (rows*cols)):.2f}%")

### post-processing

**post-processing helper (plots a gray-scale image)**

In [ ]:
def plotImage(data):
    print(data.shape)
    fig, ax = plt.subplots()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.imshow(data, cmap='gray')
    fig.tight_layout()
    plt.show()

**reconstruction of truncated data**

In [ ]:
plotImage(dataReconstruct)

**singular values**

In [ ]:
print(f"Truncation at r = {r}\nFraction of singular values: {100*np.sum(S[:r]) / np.sum(S):.2f}%")

fig, ax = plt.subplots()
ax.set_yscale('log')
ax.plot(S, 'ko')
fig.tight_layout()
plt.show()

In [ ]:
pts = 50
rs = list(set(np.logspace(np.log10(1), np.log10(min(rows, cols)), pts, dtype=int)))

# energies = np.zeros(len(rs))
singular_vals = np.zeros(len(rs))
information_retained = []
for i, sv_n in enumerate(rs):
    # energies[i] = np.sum(S[:r]**2) / np.sum(S**2)
    singular_vals[i] = np.sum(S[:sv_n]) / np.sum(S)

    reconstruction = U[:, :sv_n] @ np.diag(S[:sv_n]) @ V[:sv_n, :]
    error = np.linalg.norm(data - reconstruction, 'fro') / np.linalg.norm(data, 'fro')
    information_retained.append((1 - error))

fig, ax = plt.subplots()
# ax.semilogx(rs, energies, 'ko', label='Energy')
ax.semilogx(rs, singular_vals, 'ro', label='Singular value sum')
ax.semilogx(rs, information_retained, 'bo', label='Information retained')
ax.set_xlabel('r')
ax.set_ylabel('Fraction')
ax.set_ylim([0, 1.05])

ax.legend()


**Explore eigenvalues**

In [ ]:
from matplotlib.gridspec import GridSpec

ev_plot = 0     # eigenvalue to plot
r = 8          # truncation level

dataReconstruct = U[:, :r] @ np.diag(S[:r]) @ V[:r, :]

fig = plt.figure(figsize=(10, 8))
rows, cols = dataReconstruct.shape
gs = GridSpec(2, 2, 
              width_ratios=[cols//4, cols],  # First col width smaller for vertical plot, larger col for image
              height_ratios=[rows//4, rows], wspace=0.1, hspace=0.15)

# original image
ax_img = fig.add_subplot(gs[1, 1])
ax_img.imshow(dataReconstruct, cmap='gray', aspect='auto')  # aspect='auto' to fill the space
ax_img.axis('off')

#  horizontal slice above 
ax_h = fig.add_subplot(gs[0, 1])
ax_h.plot(V[ev_plot, :])
ax_h.set_xlim(0, cols-1)
ax_h.grid(True, axis='y')
ax_h.set_title('Horizontal Slice')

# vertical slice at the left 
ax_v = fig.add_subplot(gs[1, 0])
ax_v.plot(U[:, ev_plot], range(rows))
ax_v.invert_yaxis()  # Invert y-axis to match the image orientation
ax_v.grid(True, axis='x')
ax_v.set_title('Vertical Slice')

ax_hidden = fig.add_subplot(gs[0, 0])
ax_hidden.set_title(f"Eigenvalue {ev_plot+1} / {r}\n\n SV = {100*S[ev_plot] / S.sum():.2f}%")
ax_hidden.axis('off')

plt.tight_layout()
plt.show()